In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import gc
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
%matplotlib inline
import matplotlib.pyplot as plt
import librosa.display
import librosa
import IPython.display as ipd
from keras.models import Sequential
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets, layers, models
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import glob
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import librosa
import os
from sklearn.model_selection import train_test_split
import numpy as np
# from tqdm import tqdm
from keras import optimizers

In [ ]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

In [ ]:
emotions_ravdess={
  '01': 1,
  '02': 2,
  '03': 3,
  '04': 4,
  '05': 5,
  '06': 6,
  '07': 7,
  '08': 8
}

emotions_intensity_ravdess={
  '01': 1,
  '02': 2
}


emotions_cremad={
  'ANG':5,
  'DIS':7,
  'NEU':1,
  'HAP':3,
  'FEA':6,
  'SAD':4
}


emotions_intensity_cremad={
  'XX': 1,
  'LO': 1,
  'MD': 1,
  'HI': 2,
}

emotions_to_string={
  1:'neutral',
  2:'calm',
  3:'happy',
  4:'sad',
  5:'angry',
  6:'fearful',
  7:'disgust',
  8:'surprised'
}

emotions_intensity = {
  '01':'low',
  '02':'strong',
}

observed_emotions=[1 , 5,  3]

In [ ]:
MFCC_NUM = 40
MFCC_MAX_LEN = 2000
SAMPLING_RATE = 44100
feature_dim_1 = MFCC_NUM
feature_dim_2 = MFCC_MAX_LEN
channel = 1

In [ ]:
def extract_feature(wave, max_len=MFCC_MAX_LEN):
    mfcc = librosa.feature.mfcc(wave, n_mfcc=MFCC_NUM, sr=SAMPLING_RATE)
    if (max_len > mfcc.shape[1]):
        pad_width = max_len - mfcc.shape[1]
        mfcc = np.pad(mfcc, pad_width=((0, 0), (0, pad_width)), mode='constant')
    else:
        mfcc = mfcc[:, :max_len]
    
    return mfcc

In [ ]:
X = []
y = []

def append_X_Y(label, wave):
    y.append(label)
    mfcc = extract_feature(wave)
    X.append(mfcc)

In [ ]:
for file in glob.glob("C:\\Users\\User\\Desktop\\diploma\\diploma\\backend\\emotion-recognition\\ravdess\\Actor_*\\*.wav"):
    file_name=os.path.basename(file)
    
    label=emotions_ravdess[file_name.split("-")[2]] 
#     label=str(emotions_ravdess[file_name.split("-")[2]]) + '_' + str(emotions_intensity_ravdess[file_name.split("-")[3]]) 
    if emotions_ravdess[file_name.split("-")[2]] not in observed_emotions:
        continue
    wave, sr = librosa.load(file, mono=True, sr=44100)
    wave = wave[::3]
    append_X_Y(label, wave)
    

# for file in glob.glob("C:\\Users\\User\\Desktop\\diploma\\diploma\\backend\\emotion-recognition\\cremad\\*.wav"):
#     file_name=os.path.basename(file)
#     label=emotions_cremad[file_name.split("_")[2]] 
# #     if emotions_cremad[file_name.split("_")[2]] not in observed_emotions:
# #         continue
#     wave, sr = librosa.load(file, mono=True, sr=44100)
#     wave = wave[::3]
#     append_X_Y(label, wave)

In [ ]:
X = np.array(X)
y = np.array(y)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.25, random_state=True, shuffle=True)

In [ ]:
X_train.shape[1]

In [ ]:
# for testing new model 1

model = models.Sequential()
model.add(layers.Conv2D(64, (3, 3), activation='relu', input_shape=(50, 259, channel)))
model.add(layers.Conv2D(128, (3, 3), activation='relu', strides=(1,1), padding=0))
model.add(layers.Conv2D(256, (3, 3), activation='relu', strides=(1,1), padding=0))
model.add(layers.Conv2D(512, (3, 3), activation='relu', strides=(1,1), padding=0))
model.add(layers.Conv2D(512, (2, 2), activation='relu', strides=(1,1), padding=0))
model.add(layers.MaxPool2D((2, 2)))

In [ ]:
# for testing new model 2

model = Sequential()
model.add(Conv1D(256, 8, padding='same',input_shape=(X_train.shape[1],1)))
model.add(Activation('relu'))
model.add(Conv1D(256, 8, padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))
model.add(MaxPooling1D(pool_size=(8)))
model.add(Conv1D(128, 8, padding='same'))
model.add(Activation('relu'))
model.add(Conv1D(128, 8, padding='same'))
model.add(Activation('relu'))
model.add(Conv1D(128, 8, padding='same'))
model.add(Activation('relu'))
model.add(Conv1D(128, 8, padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))
model.add(MaxPooling1D(pool_size=(8)))
model.add(Conv1D(64, 8, padding='same'))
model.add(Activation('relu'))
model.add(Conv1D(64, 8, padding='same'))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(14)) # Target class number
model.add(Activation('softmax'))
opt = tf.keras.optimizers.RMSprop(lr=0.00001, decay=1e-6)

In [ ]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(X_train.shape[1], feature_dim_1, feature_dim_2, channel)))
model.add(layers.MaxPooling2D((3, 3)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((3, 3)))
model.add(layers.Conv2D(120, (3, 3), activation='relu'))

In [ ]:
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='sgd',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

history = model.fit(X_train, y_train, epochs=10, validation_split=0.25)

In [ ]:
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')

test_loss, test_acc = model.evaluate(X_test, y_test, verbose=2)

In [ ]:
print(test_acc)

In [ ]:
import pickle

In [ ]:
filename = 'cnn_3_emotions_73.pkl'
pickle.dump(model, open(filename, 'wb'))

In [ ]:
model.save("C:\\Users\\User\\Desktop\\diploma\\diploma\\backend\\emotion-recognition\\")

In [ ]:
newFile = "C:/Users/User/Desktop/diploma/diploma/backend/speech-diarization/try/17_test.wav"

In [ ]:
wave, sr = librosa.load(newFile, mono=True, sr=44100)
wave = wave[::3]
mfcc = wav2mfcc(wave)
X_test = mfcc.reshape(1, feature_dim_1, feature_dim_2, channel)
preds = model.predict(X_test)[0]
classes_x=np.argmax(preds,axis=0)
emotions_to_string[classes_x]

In [ ]:
Emotion_labels = ['neutral', 'calm', 'happy', 'sad', 'angry', 'fearful', 'disgust', 'surprised']
cnt_emotion_labels = [0, 0, 0, 0, 0, 0, 0, 0]

emotions_ravdess={
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fearful',
  '07':'disgust',
  '08':'surprised'
}


for file in glob.glob("C:\\Users\\User\\Desktop\\diploma\\diploma\\backend\\emotion-recognition\\ravdess\\Actor_*\\*.wav"):
    file_name=os.path.basename(file)
    cnt_emotion_labels[Emotion_labels.index(emotions_ravdess[file_name.split("-")[2]])]=cnt_emotion_labels[Emotion_labels.index(emotions_ravdess[file_name.split("-")[2]])]+1
  

plt.bar(Emotion_labels, cnt_emotion_labels) 
plt.show()

In [ ]:
Emotion_labels = ['ANG', 'DIS', 'NEU', 'HAP', 'FEA', 'SAD']
cnt_emotion_labels = [0, 0, 0, 0, 0, 0]


for file in glob.glob("C:\\Users\\User\\Desktop\\diploma\\diploma\\backend\\emotion-recognition\\cremad\\*.wav"):
    file_name=os.path.basename(file)
    cnt_emotion_labels[Emotion_labels.index(file_name.split("_")[2])]=cnt_emotion_labels[Emotion_labels.index(file_name.split("_")[2])]+1
    

plt.bar(Emotion_labels, cnt_emotion_labels) 
plt.show()